In [0]:
-- SALES HISTORY --

with sales_history as (
  select a.flightkey, a.dtg, a.unt_net as sales, to_date(a.charge_dt, 'yyyyMMdd') as charge_dt, to_date(b.onsale_dt, 'yyyyMMdd') as first_date, least(to_date(left(a.flightkey, 8), 'yyyyMMdd'), to_date(current_date(), 'yyyyMMdd')) as last_date, date_diff(to_date(left(a.flightkey, 8), 'yyyyMMdd'), b.onsale_dt) as sale_length, substring(a.flightkey, 9, 6) as sector, to_date(left(a.flightkey, 8), 'yyyyMMdd') as flight_dt, b.flight_std as flight_time
  from data_experience_commercial.cbt_1423_rtsuite.master_uat a
  left join data_experience_commercial.cbt_0923_segmentfinder.dimensions_history b 
  on a.flightkey=b.flightkey
  where 1=1
  and a.chargeproduct = 'Ticket'
  and to_date(a.charge_dt, 'yyyyMMdd') <= least(to_date(left(a.flightkey, 8), 'yyyyMMdd'), to_date(current_date(), 'yyyyMMdd'))
  and to_date(a.charge_dt, 'yyyyMMdd') >= to_date(b.onsale_dt, 'yyyyMMdd')
  and date_diff(to_date(a.charge_dt, 'yyyyMMdd'), b.onsale_dt) >= 7
  and to_date(left(a.flightkey, 8), 'yyyyMMdd') >= '2024-01-01'
  and to_date(left(a.flightkey, 8), 'yyyyMMdd') <= '2024-06-30'
),

flightkeydtg_sales as (
  select flightkey, charge_dt, first(sector) as sector, first(flight_time) as flight_time, first(flight_dt) as flight_dt, sum(sales) as sales, first(first_date) as first_date, first(last_date) as last_date
  from sales_history
  group by flightkey, charge_dt
),

flightkey_sales as (
  select flightkey, first(first_date) as first_date, first(last_date) as last_date
  from flightkeydtg_sales
  group by flightkey
),

datespine_sales as (
  select flightkey, explode(sequence(first_date, last_date, interval 1 day)) as calender_date
  from flightkey_sales
),

filled_sales as (
select a.flightkey, a.calender_date as date, ifnull(b.sales, 0) as sales, sum(ifnull(b.sales, 0)) over (partition by a.flightkey order by a.calender_date asc rows between unbounded preceding and current row) as cumulative_sales, b.sector, b.flight_dt, b.flight_time, b.charge_dt
from datespine_sales a
left join flightkeydtg_sales b
on a.flightkey=b.flightkey and a.calender_date=b.charge_dt
where 1=1
order by a.calender_date),

offsets as (
    select explode(array(7, 14, 21, 28)) as days_lag
),

potential_matches AS (
  select a.flightkey as flightkey, a.flight_dt as current_flight_date, a.sector, b.flightkey as reference_flightkey, b.flight_std AS hist_flight_time, c.days_lag, abs((hour(a.flight_time)*60 + minute(a.flight_time)) - (hour(b.flight_std)*60 + minute(b.flight_std))) as time_diff_mins
  from filled_sales a
  cross join offsets c
  inner join data_experience_commercial.cbt_0923_segmentfinder.dimensions_history b 
  on a.sector = b.sector and b.flight_dt = date_sub(a.flight_dt, c.days_lag)
),

ranked_matches as (
  select *, 
  case 
    when time_diff_mins = 0 then 1
    when time_diff_mins <= 60 then 2
    when time_diff_mins <= 240 then 3
    else 4
  end as match_quality_rank
  from potential_matches
),

match_selection as (
  select * from ranked_matches
  qualify row_number() over (partition by flightkey, days_lag order by match_quality_rank asc, time_diff_mins asc) = 1
),

equivalent_flights as (
 select flightkey,
    max(case when days_lag = 7 then reference_flightkey end) as eq_flightkey_7,
    max(case when days_lag = 14 then reference_flightkey end) as eq_flightkey_14,
    max(case when days_lag = 21 then reference_flightkey end) as eq_flightkey_21,
    max(case when days_lag = 28 then reference_flightkey end) as eq_flightkey_28
from match_selection
group by flightkey
),

double_lags as (
  select a.flightkey, a.date, a.sales, a.cumulative_sales, ifnull(s7.sales, 0) as SF7C7, ifnull(s14.sales, 0) as SF14C14, ifnull(s21.sales, 0) as SF21C21, ifnull(s28.sales, 0) as SF28C28
  from filled_sales a 
  left join equivalent_flights map 
  on a.flightkey = map.flightkey
  left join filled_sales s7
  on s7.flightkey = map.eq_flightkey_7
  and s7.charge_dt = date_sub(a.charge_dt, 7)
  left join filled_sales s14
  on s14.flightkey = map.eq_flightkey_14
  and s14.charge_dt = date_sub(a.charge_dt, 14)
  left join filled_sales s21
  on s21.flightkey = map.eq_flightkey_21
  and s21.charge_dt = date_sub(a.charge_dt, 21)
  left join filled_sales s28
  on s28.flightkey = map.eq_flightkey_28
  and s28.charge_dt = date_sub(a.charge_dt, 28)
),

-- PRICE HISTORY --

price_history as (
  select a.segment, a.standardprice, a.currencycode, to_date(a.when, 'yyyyMMdd') as charge_dt, to_date(b.onsale_dt, 'yyyyMMdd') as first_date, least(to_date(left(a.segment, 8), 'yyyyMMdd'), to_date(current_date(), 'yyyyMMdd')) as last_date
  from data_prod.silver_sanezdb.priceinspection a
  left join data_experience_commercial.cbt_0923_segmentfinder.dimensions_history b 
  on a.segment=b.flightkey
  where 1=1
  and to_date(a.when, 'yyyyMMdd') <= least(to_date(left(a.segment, 8), 'yyyyMMdd'), to_date(current_date(), 'yyyyMMdd'))
  and to_date(a.when, 'yyyyMMdd') >= to_date(b.onsale_dt, 'yyyyMMdd')
  and to_date(left(a.segment, 8), 'yyyyMMdd') >= '2024-04-01'
  and to_date(left(a.segment, 8), 'yyyyMMdd') <= '2024-06-30'
),

segmentdtg_price as (
  select segment, charge_dt, avg(standardprice) as price, first(currencycode) as currency, first(first_date) as first_date, first(last_date) as last_date
  from price_history
  group by segment, charge_dt
),

segment_price as (
  select segment, first(first_date) as first_date, first(last_date) as last_date
  from segmentdtg_price
  group by segment
),

datespine_price as (
  select segment, explode(sequence(first_date, last_date, interval 1 day)) as calender_date
  from segment_price
),

body_price as (
  select a.segment, a.calender_date, b.price, b.currency
  from datespine_price a
  left join segmentdtg_price b
  on a.segment=b.segment and a.calender_date=b.charge_dt
),

ffill_price as (
  select segment, calender_date, last_value(price) ignore nulls over (partition by segment order by calender_date rows between unbounded preceding and current row) as price_ffill, last_value(currency) ignore nulls over (partition by segment order by calender_date rows between unbounded preceding and current row) as currency_ffill
  from body_price
),

filled_price as (
  select segment, calender_date as date, coalesce(price_ffill, first_value(price_ffill) ignore nulls over (partition by segment order by calender_date rows between current row and unbounded following)) as price_filled, coalesce(currency_ffill, first_value(currency_ffill) ignore nulls over (partition by segment order by calender_date rows between current row and unbounded following)) as currency_filled
  from ffill_price
),

-- PROFILE HISTORY --

profile_history as (
  select a.flightkey, to_date(a.rms_profile_changedttm, 'yyyyMMdd') as date, least(to_date(left(a.flightkey, 8), 'yyyyMMdd'), to_date(current_date(), 'yyyyMMdd')) as last_date, least(to_date(b.onsale_dt, 'yyyyMMdd'), to_date(a.rms_profile_changedttm, 'yyyyMMdd'))  as first_date, 
  a.rms_profile, a.rms_profile_tgt, b.ty_capacity
  from data_experience_commercial.cbt_0923_segmentfinder.rms_history a
  left join data_experience_commercial.cbt_0923_segmentfinder.dimensions_history b
  on a.flightkey = b.flightkey
  where 1=1
  and to_date(left(a.flightkey, 8), 'yyyyMMdd') >= '2024-04-01'
  and to_date(left(a.flightkey, 8), 'yyyyMMdd') <= '2024-06-30'
),

flightkey_profiles_ as (
  select flightkey, first(first_date) as first_date, first(last_date) as last_date
  from profile_history
  group by flightkey
),

flightkey_profiles as (
  select * from flightkey_profiles_ where first_date < last_date
),

datespine_profiles as (
  select flightkey, explode(sequence(first_date, last_date, interval 1 day)) as calender_date
  from flightkey_profiles
),

body_profiles as (
  select a.flightkey, a.calender_date, b.rms_profile, b.rms_profile_tgt
  from datespine_profiles a
  left join profile_history b
  on a.flightkey=b.flightkey and a.calender_date=b.date
),

ffill_profiles as (
  select flightkey, calender_date, last_value(rms_profile) ignore nulls over (partition by flightkey order by calender_date rows between unbounded preceding and current row) as rms_profile_ffill, rms_profile_tgt
  from body_profiles
),

filled_profiles_ as (
  select flightkey, calender_date as date, coalesce(rms_profile_ffill, first_value(rms_profile_ffill) ignore nulls over (partition by flightkey order by calender_date rows between current row and unbounded following)) as profile_filled, rms_profile_tgt as profile_tgt
  from ffill_profiles
),

filled_profiles as (
  select flightkey, date, first(profile_filled) as profile_filled, first(profile_tgt) as profile_tgt
  from filled_profiles_ 
  group by flightkey, date
),

-- CAPACITY HISTORY -- 

cap_history as (
  select a.flightkey, to_date(left(a.flightkey, 8), 'yyyyMMdd') as flight_dt, to_date(a.lidcapupdatedtime, 'yyyyMMdd') as charge_dt,  avg(a.capacity) as capacity, avg(a.lid) as lid, 
  first(to_date(b.onsale_dt, 'yyyyMMdd')) as onsale_dt, least(to_date(left(a.flightkey, 8), 'yyyyMMdd'), to_date(current_date(), 'yyyyMMdd')) as last_date
  from data_prod.silver_sanezdb.lidcaprecords a
  left join data_experience_commercial.cbt_0923_segmentfinder.dimensions_history b
  on a.flightkey=b.flightkey
  where 1=1
  and to_date(left(a.flightkey, 8), 'yyyyMMdd') >= '2024-04-01'
  and to_date(left(a.flightkey, 8), 'yyyyMMdd') <= '2024-06-30'
  group by a.flightkey, a.lidcapupdatedtime
),

flightkey_caphistory_ as (
  select flightkey, first(last_date) as last_date, least(first(onsale_dt), min(charge_dt)) as first_date
  from cap_history
  group by flightkey
),

flightkey_caphistory as (
  select * from flightkey_caphistory_ where first_date < last_date
),

datespine_capacity as (
  select flightkey, explode(sequence(first_date, last_date, interval 1 day)) as calender_date
  from flightkey_caphistory
),

body_capacity as (
  select a.flightkey, a.calender_date, b.capacity, b.lid
  from datespine_capacity a
  left join cap_history b
  on a.flightkey=b.flightkey and a.calender_date=b.charge_dt
),

ffill_cap as (
  select flightkey, calender_date, last_value(capacity) ignore nulls over (partition by flightkey order by calender_date rows between unbounded preceding and current row) as capacity, last_value(lid) ignore nulls over (partition by flightkey order by calender_date rows between unbounded preceding and current row) as lid
  from body_capacity
),

filled_cap as (
  select flightkey, calender_date, first_value(capacity) ignore nulls over (partition by flightkey order by calender_date rows between current row and unbounded following) as capacity, first_value(lid) ignore nulls over (partition by flightkey order by calender_date rows between current row and unbounded following) as lid
  from ffill_cap
),

-- ROS HISTORY --
 
ros_history as (
  select segment, to_date(datestamp, 'yyyyMMdd') as date, date_diff(to_date(left(segment, 8), 'yyyyMMdd'), to_date(datestamp, 'yyyyMMdd')) as dtg, (username), (breakpointprofile)
  from data_prod.silver_sanezdb.rmsbreakpointhistory
  where 1=1
  and ((username = 'ROS Increase' and breakpointprofile like '%GB%') or (username = 'ROS Decrease' and breakpointprofile like '%GB%') or (username = 'ROSIncreaseFB') or (username = 'ROSReduceFB') or (username = 'ROSIncreaseBP')  or (username = 'ROSReduceBP'))
  and to_date(left(segment, 8), 'yyyyMMdd') >= '2024-04-01'
  and to_date(left(segment, 8), 'yyyyMMdd') <= '2024-06-30'
),

grouped_ros_history as (
  select segment, dtg, first(date) as date, first(username) as username, first(breakpointprofile) as breakpointprofile
  from ros_history
  group by segment, dtg
),

-- COMBINED HISTORY --


combined_history  as (
  select a.flightkey, to_date(left(a.flightkey, 8), 'yyyyMMdd') as flight_dt, a.date as charge_dt, date_diff(to_date(left(a.flightkey, 8), 'yyyyMMdd'), a.date) as dtg, a.sales, a.cumulative_sales, a.SF7C7, a.SF14C14, a.SF21C21, a.SF28C28, b.price_filled, b.currency_filled, c.profile_filled, d.capacity, d.lid, 
  e.username, e.breakpointprofile, f.sector, f.route, f.parentregion as region, f.routetype, f.flight_std as flight_time, f.flight_dow
  from filled_price b
  left join double_lags a
  on b.segment = a.flightkey and b.date = a.date
  left join filled_profiles c
  on a.flightkey = c.flightkey and a.date = c.date
  left join filled_cap d
  on a.flightkey = d.flightkey and a.date = d.calender_date
  left join grouped_ros_history e
  on a.flightkey = e.segment and a.date = e.date
  left join data_experience_commercial.cbt_0923_segmentfinder.dimensions_history f
  on a.flightkey = f.flightkey
),

lag_cols as (
  select *, greatest(0, coalesce((lag(sales, 1) over w), 0)) as sales_lag_1, greatest(0, coalesce((lag(sales, 2) over w), 0)) as sales_lag_2, greatest(0, coalesce((lag(sales, 3) over w), 0)) as sales_lag_3, greatest(0, coalesce((lag(sales, 7) over w), 0)) as lag_sales_7, greatest(0, coalesce((lag(sales, 14) over w), 0)) as lag_sales_14, greatest(0, coalesce((lag(sales, 21) over w), 0)) as lag_sales_21, greatest(0, coalesce((lag(sales, 28) over w), 0)) as lag_sales_28, greatest(0, coalesce((lead(sales, 1) over w), 0)) as sales_lead_1, greatest(0, coalesce((lead(sales, 2) over w), 0)) as sales_lead_2, greatest(0, coalesce((lead(sales, 3) over w), 0)) as sales_lead_3, coalesce((lag(price_filled, 1) over w), 0) as price_lag_1, coalesce((lead(price_filled, 1) over w), 0) as price_lead_1
  from combined_history
  window w as (partition by flightkey order by charge_dt)
),

treatment_outcome as (
  select *, ln(price_lead_1) - ln(price_lag_1) as treatment_delta, ln(sales_lead_1 + sales_lead_2 + sales_lead_3 + 1) - ln(sales_lag_1 + sales_lag_2 + sales_lag_3 + 1) as outcome_delta, ln(sales_lag_1 + sales_lag_2 + sales_lag_3 + 1) as log_sales_pre, ln(sales_lead_1 + sales_lead_2 + sales_lead_3 + 1) as log_sales_post, ln(price_lead_1) as log_price_post, ln(price_lag_1) as log_price_pre
  from lag_cols where username like '%ROS%'
),

filtered as (
  select * from treatment_outcome
  where (username like '%Increase%' and treatment_delta > 0) or (username like '%Decrease%' and treatment_delta < 0) or (username like '%Reduce%' and treatment_delta < 0)
),

standardized_profiles as (
  select *, 
    case 
      when length(profile_filled) = 2 then 'ALPHA' else 'NUMERIC' 
    end as profile_type,
    case 
      when profile_filled like '%:%' then 0
      when profile_filled like '%_%' then cast(split(profile_filled, '_')[0] as int)
      else null
    end as x100,
    case 
      when profile_filled like '%:%' then cast(split(split(profile_filled, ',')[0], ':')[1] as int)
      when profile_filled like '%_%' then cast(split(profile_filled, '_')[1] as int)
      else null
    end as x60,
    case 
      when profile_filled like '%:%' then cast(split(split(profile_filled, ',')[1], ':')[1] as int)
      when profile_filled like '%_%' then cast(split(profile_filled, '_')[2] as int)
      else null
    end as x20,
    case 
      when profile_filled like '%:%' then cast(split(profile_filled, ',')[2] as int)
      when profile_filled like '%_%' then cast(split(profile_filled, '_')[3] as int)
      else null
    end as x00
  from filtered
),

numeric_calc_stage_1 as (
  select *,
    case when profile_type = 'NUMERIC' then (x60-x100)/(x00-x100) end as x1,
    case when profile_type = 'NUMERIC' then (x20-x100)/(x00-x100) end as x2
  from standardized_profiles
),

numeric_calc_stage_2 as (
  select *,
    case when x1 is not null then log(log((1-power(x1, 4))/0.6)/log((1-power(x2, 4))/0.2))/log(x1/x2) end as b_const
  from numeric_calc_stage_1
),

numeric_final as (
  select *, log((1-power(x1, 4))/0.6)/power(x1, b_const) as a_const, (dtg - x100)/(x00 - x100) as x_norm
  from numeric_calc_stage_2
),

combined_logic as (
  select base.*,
    nanvl((exp(-base.a_const * power(base.x_norm, base.b_const))) * (1-power(base.x_norm, 4)), 1) as numeric_prof_LF,
    coalesce(
      case 
        when PA.LoadFactor IS null then PB.LoadFactor 
        else PA.LoadFactor + ((PB.LoadFactor - PA.LoadFactor) * (base.DTG / 
            case 
              when (f.MxDTOG + c.ActiveSellOutDaysToGo) > ((f.MxDTOG + c.ActiveSellOutDaysToGo) / c.ProfileShapeShiftHorizontal) 
              then ((f.MxDTOG + c.ActiveSellOutDaysToGo) / c.ProfileShapeShiftHorizontal) 
              else (f.MxDTOG + c.ActiveSellOutDaysToGo) 
            end)) 
      end, 0) * c.ProfileShapeShiftVertical as alpha_prof_LF
  from numeric_final base
  
  left join data_prod.silver_stream_sanezdb.segmentdetails c
  on base.flightkey = c.Segment
  and base.profile_type = 'ALPHA' 
  left join data_experience_commercial.team_pricing_revenue_mgmt.oldprofiledata PA
  on base.profile_filled = PA.SellingProfileId     
  and case 
        when (base.DTG - c.ActiveSellOutDaysToGo) <= -1 then -1 
        else (base.DTG - c.ActiveSellOutDaysToGo) 
      end = PA.DaysToGo
  and base.profile_type = 'ALPHA'
  left join data_experience_commercial.team_pricing_revenue_mgmt.oldprofiledata PB
  on base.profile_filled = PB.SellingProfileId  
  and case 
        when (base.DTG - c.ActiveSellOutDaysToGo) <= -1 then -1 
        else round((base.DTG - c.ActiveSellOutDaysToGo) * c.ProfileShapeShiftHorizontal, 0)
      end = PB.DaysToGo
  and base.profile_type = 'ALPHA'
  left join (
      select sellingprofileId, max(daystogo) as MxDTOG
      from data_experience_commercial.team_pricing_revenue_mgmt.oldprofiledata
      group by sellingprofileId) f 
  on base.profile_filled = f.sellingprofileId   
  and base.profile_type = 'ALPHA'
),

effective_prof_LF as (
  select *,
  case 
    when profile_type = 'NUMERIC' then numeric_prof_LF
    else alpha_prof_LF
  end as Effective_Profile_LF
  from combined_logic
),

effective_target_seats as (
  select *, round((Effective_Profile_LF*lid), 0) as effective_target_seats
  from effective_prof_LF
),

markets as (
    select sector, first(sectormarketpair) as sectormarketpair
    from data_experience_commercial.cbt_0899_competitordashboard.competitoralerts_history
    group by sector
),

final_combined_ as (
  select a.flightkey, a.flight_dt, a.flight_time, a.flight_dow, a.charge_dt, a.dtg, a.sales, a.cumulative_sales, (a.lid-a.cumulative_sales) as seats_remaining, a.price_filled, a.currency_filled, a.sector, b.sectormarketpair as market_sector, a.route, a.region, a.routetype, a.capacity, a.lid, a.treatment_delta, a.outcome_delta, a.log_sales_pre, a.log_sales_post, a.log_price_pre, a.log_price_post, a.effective_target_seats, (a.cumulative_sales-a.effective_target_seats) as Z_instrument, coalesce(a.lag_sales_7, 0) as lag_sales_7, coalesce(a.lag_sales_14, 0) as lag_sales_14, coalesce(a.lag_sales_21, 0) as lag_sales_21, coalesce(a.lag_sales_28, 0) as lag_sales_28, a.SF7C7, a.SF14C14, a.SF21C21, a.SF28C28
  from effective_target_seats a
  inner join markets b
  on a.sector=b.sector
),

final_combined as (
  select * from final_combined_ 
  where 1=1
  and ((Z_instrument > 0 and treatment_delta > 0) or (Z_instrument < 0 and treatment_delta < 0))
),

comp as (
  select to_date(observation_date, 'yyyyMMdd') as observation_date, origin, destination, sector, carrier, --b.carriertype
  to_date(outbound_departure_date, 'yyyyMMdd') as flight_dt, cast(concat(date_format(to_date(outbound_departure_date, 'yyyyMMdd'), 'yyyy-MM-dd'), ' ', outbound_departure_time) as timestamp) as comp_flight_time, days_to_go, currency,
  case
    when is_one_way = 1 then price_inc
    else
      case
        when price_outbound is null then price_inc/2
        else price_outbound + (coalesce(tax, 0)/2)
      end
  end as comp_price
  from data_prod.silver_curated_infare.infaredata
  where 1=1
  and to_date(outbound_departure_date, 'yyyyMMdd') >= '2024-04-01'
  and to_date(outbound_departure_date, 'yyyyMMdd') <= '2024-06-30' 
),

comp_curated as (
  select observation_date, sector, carrier, flight_dt, comp_flight_time, first(days_to_go) as dtg, avg(comp_price) as comp_price, first(currency) as currency--, first(carriertype) as carrier_type
  from comp group by observation_date, sector, carrier, flight_dt, comp_flight_time
),

comp_curated_market as (
  select a.*, b.sectormarketpair as market_sector
  from comp_curated a
  inner join markets b
  on a.sector=b.sector
),

optionality as (
  select a.flightkey, a.region, a.routetype, a.flight_dow, a.dtg, b.comp_flight_time, b.comp_price, b.currency, (unix_timestamp(b.comp_flight_time) - unix_timestamp(a.flight_time))/3600 as time_diff, b.comp_price/a.price_filled as price_ratio, cast(substring(b.comp_flight_time, 12, 2) as int) as comp_flight_hod
  from final_combined a 
  left join comp_curated_market b
  on a.market_sector = b.market_sector and a.flight_dt = b.flight_dt and a.dtg=b.dtg
  where a.currency_filled = b.currency
),

final_RPS as (
  select flightkey, max(ty_ticketrev_net) as final_rev
  from data_experience_commercial.cbt_0923_segmentfinder.ticket_history
  where 1=1
  and to_date(left(flightkey, 8), 'yyyyMMdd') >= '2024-04-01'
  and to_date(left(flightkey, 8), 'yyyyMMdd') <= '2024-06-30'
  group by flightkey
),

hourly_aggregates as (
    select a.parentregion, a.routetype, a.flight_dow, a.flight_hod, avg(b.final_rev)/avg(a.ty_capacity) as final_RPS
    from data_experience_commercial.cbt_0923_segmentfinder.dimensions_history a
    left join final_RPS b
    on a.flightkey = b.flightkey
    and a.flight_dt between '2024-04-01' and '2024-06-30'
    group by a.parentregion, a.routetype, a.flight_dow, a.flight_hod
),

scaled_quality_scores as (
    select *, min(final_RPS) over (partition by parentregion, routetype) as min_rps, max(final_RPS) over (partition by parentregion, routetype) as max_rps,
        case
            when max(final_RPS) over (partition by parentregion, routetype) = min(final_RPS) over (partition by parentregion, routetype) 
            then 0.5 -- Default value if all RPS are identical
            else 
                ((final_RPS - min(final_RPS) over (partition by parentregion, routetype)) / (max(final_RPS) over (partition by parentregion, routetype) - min(final_RPS) over (partition by parentregion, routetype))) + 1
        end as time_quality_score
    from 
        hourly_aggregates
),

time_score as (
  select a.*, b.time_quality_score
  from optionality a
  left join scaled_quality_scores b
  on a.region = b.parentregion and a.routetype = b.routetype and a.flight_dow = b.flight_dow and a.comp_flight_hod = b.flight_hod),


optionality_score as (
  select flightkey, dtg, sum(time_quality_score/((power(price_ratio, 2))*(power((abs(time_diff) + 1), 2)))) as optionality_score, sum(time_quality_score/((price_ratio)*(abs(time_diff)+1))) as linear_optionality_score 
  from time_score
  group by flightkey, dtg
),

combined_with_metrics as (
  select a.*, coalesce(b.optionality_score, 0) as optionality_score, coalesce(b.linear_optionality_score, 0) as linear_optionality_score
  from final_combined a
  left join optionality_score b
  on a.flightkey=b.flightkey and a.dtg=b.dtg
),

flight_markets as (
  select a.flightkey, b.sectormarketpair as market_sector
  from data_experience_commercial.cbt_0923_segmentfinder.dimensions_history a
  left join markets b
  on a.sector = b.sector
  where 1=1
  and to_date(left(a.flightkey, 8), 'yyyyMMdd') >= '2024-04-01'
  and to_date(left(a.flightkey, 8), 'yyyyMMdd') <= '2024-06-30'
),

flight_market_prices as (
  select a.segment, to_date(left(a.segment, 8), 'yyyyMMdd') as flight_dt, a.date as charge_dt, a.price_filled, b.market_sector
  from filled_price a
  left join flight_markets b
  on a.segment = b.flightkey
),

neighbours as (
  select a.flightkey, a.dtg, a.flight_time, a.price_filled, a.region, a.routetype, a.flight_dow, b.segment as neighbour_flightkey, b.price_filled as neighbour_price, b.price_filled/a.price_filled as neighbour_price_ratio
  from combined_with_metrics a
  left join flight_market_prices b
  on a.market_sector = b.market_sector and a.flight_dt = b.flight_dt and a.charge_dt = b.charge_dt
  where a.flightkey <> b.segment),

neighbour_time as (
  select a.*, b.flight_std as neighbor_flight_time, (unix_timestamp(a.flight_time) - unix_timestamp(b.flight_std))/3600 as neighbour_time_diff, cast(substring(b.flight_std, 12, 2) as int) as neighbour_flight_hod
  from neighbours a
  left join data_experience_commercial.cbt_0923_segmentfinder.dimensions_history b
  on a.neighbour_flightkey=b.flightkey
),

neighbour_time_score as (
  select a.*, b.time_quality_score as neighbour_time_quality_score
  from neighbour_time a
  left join scaled_quality_scores b
  on a.region = b.parentregion and a.routetype = b.routetype and a.flight_dow = b.flight_dow and a.neighbour_flight_hod = b.flight_hod
),

internal_optionality_score as (
  select flightkey, dtg, sum(neighbour_time_quality_score/((power(neighbour_price_ratio, 2))*(power((abs(neighbour_time_diff) + 1), 2)))) as internal_optionality_score, sum(neighbour_time_quality_score/((neighbour_price_ratio)*(abs(neighbour_time_diff)+1))) as linear_internal_optionality_score
  from neighbour_time_score
  group by flightkey, dtg
),

total_optionality as (
  select a.*, a.optionality_score + coalesce(b.internal_optionality_score, 0) as total_optionality_score, a.linear_optionality_score + coalesce(b.linear_internal_optionality_score, 0) as total_linear_optionality_score, left(sector, 3) as origin_airport, right(sector, 3) as destination_airport,
      case 
        when a.dtg <= 7 then 1
        when a.dtg >= 8 and a.dtg <= 14 then 2
        when a.dtg >= 15 and a.dtg <= 28 then 3
        when a.dtg >= 29 and a.dtg <= 42 then 4
        when a.dtg >= 43 and a.dtg <= 56 then 5
        when a.dtg >= 57 and a.dtg <= 84 then 6
        when a.dtg >= 85 and a.dtg <= 112 then 7
        when a.dtg >= 113 and a.dtg <= 168 then 8
        when a.dtg >= 169 and a.dtg <= 224 then 9
        when a.dtg >= 225 and a.dtg <= 252 then 10
      end as dtgperiod
  from combined_with_metrics a
  left join internal_optionality_score b
  on a.flightkey=b.flightkey and a.dtg=b.dtg
),

business_penetration as (
  select first_flightkey, date_format(to_date(left(first_flightkey, 8), 'yyyyMMdd'), 'E') as DoW_first_flight, date_diff(to_date(left(first_flightkey, 8), 'yyyyMMdd'), to_date(booking_dttm, 'yyyyMMdd')) as dtg, left(first_sector, 3) as origin_airport, right(first_sector, 3) as destination_airport, bl_flag
  from data_experience_commercial.cbt_1470_reservationattributes.augmented
  where to_date(left(first_flightkey, 8), 'yyyyMMdd') between '2024-04-01' and '2024-06-30'
),

origin_airport_BP as (
    select origin_airport, avg(case when bl_flag = 'Business' then 1.0 else 0.0 end) as origin_bp
    from business_penetration
    group by origin_airport
),

origin_airport_BP_by_DoW as (
    select origin_airport, DoW_first_flight, avg(case when bl_flag = 'Business' then 1.0 else 0.0 end) as origin_bp_DoW
    from business_penetration
    group by origin_airport, DoW_first_flight
),

origin_airport_BP_by_DoW_DTG as (
    select origin_airport, DoW_first_flight,
      case 
        when dtg <= 7 then 1
        when dtg >= 8 and dtg <= 14 then 2
        when dtg >= 15 and dtg <= 28 then 3
        when dtg >= 29 and dtg <= 42 then 4
        when dtg >= 43 and dtg <= 56 then 5
        when dtg >= 57 and dtg <= 84 then 6
        when dtg >= 85 and dtg <= 112 then 7
        when dtg >= 113 and dtg <= 168 then 8
        when dtg >= 169 and dtg <= 224 then 9
        when dtg >= 225 and dtg <= 252 then 10
      end as dtgperiod,
    avg(case when bl_flag = 'Business' then 1.0 else 0.0 end) as origin_bp_DoW_DTG
    from business_penetration
    group by origin_airport, DoW_first_flight, dtgperiod
),

destination_airport_BP as (
    select destination_airport, avg(case when bl_flag = 'Business' then 1.0 else 0.0 end) as destination_bp
    from business_penetration
    group by destination_airport
),

destination_airport_BP_by_DoW as (
    select destination_airport, DoW_first_flight, avg(case when bl_flag = 'Business' then 1.0 else 0.0 end) as destination_bp_DoW
    from business_penetration
    group by destination_airport, DoW_first_flight
),

destination_airport_BP_by_DoW_DTG as (
    select destination_airport, DoW_first_flight,
      case 
        when dtg <= 7 then 1
        when dtg >= 8 and dtg <= 14 then 2
        when dtg >= 15 and dtg <= 28 then 3
        when dtg >= 29 and dtg <= 42 then 4
        when dtg >= 43 and dtg <= 56 then 5
        when dtg >= 57 and dtg <= 84 then 6
        when dtg >= 85 and dtg <= 112 then 7
        when dtg >= 113 and dtg <= 168 then 8
        when dtg >= 169 and dtg <= 224 then 9
        when dtg >= 225 and dtg <= 252 then 10
      end as dtgperiod,
    avg(case when bl_flag = 'Business' then 1.0 else 0.0 end) as destination_bp_DoW_DTG
    from business_penetration
    group by destination_airport, DoW_first_flight, dtgperiod
),

total_BP as (
  select a.*, b.origin_bp, c.destination_bp, b.origin_bp*c.destination_bp as combined_bp, d.origin_bp_DoW, e.destination_bp_DoW, d.origin_bp_DoW*e.destination_bp_DoW as combined_bp_DoW, f.origin_bp_DoW_DTG, g.destination_bp_DoW_DTG, f.origin_bp_DoW_DTG*g.destination_bp_DoW_DTG as combined_bp_DoW_DTG
  from total_optionality a
  left join origin_airport_BP b
  on a.origin_airport=b.origin_airport
  left join destination_airport_BP c
  on a.destination_airport=c.destination_airport
  left join origin_airport_BP_by_DoW d
  on a.origin_airport=d.origin_airport and a.flight_dow=d.DoW_first_flight
  left join destination_airport_BP_by_DoW e
  on a.destination_airport=e.destination_airport and a.flight_dow=e.DoW_first_flight
  left join origin_airport_BP_by_DoW_DTG f
  on a.origin_airport=f.origin_airport and a.flight_dow=f.DoW_first_flight and a.dtgperiod=f.dtgperiod
  left join destination_airport_BP_by_DoW_DTG g
  on a.destination_airport=g.destination_airport and a.flight_dow=g.DoW_first_flight and a.dtgperiod=g.dtgperiod 
  ),

curated_BP as (
  select flightkey, flight_dt, flight_time, charge_dt, dayofweek(flight_dt) as flight_dow, dayofweek(charge_dt) AS charge_dow, dayofmonth(flight_dt) as flight_dom, dayofmonth(charge_dt) as charge_dom, month(flight_dt) as flight_mth, month(charge_dt) as charge_mth, dtg, route, region, routetype,
  lid, cumulative_sales, seats_remaining, round((cumulative_sales/capacity), 2) as Loadfactor, effective_target_seats, Z_instrument, round(treatment_delta, 6) as treatment_delta, round(outcome_delta, 6) as outcome_delta, round(log_sales_pre, 6) as log_sales_pre, lag_sales_7, lag_sales_14, lag_sales_21, lag_sales_28, SF7C7, SF14C14, SF21C21, SF28C28, round(total_linear_optionality_score, 6) as total_linear_optionality_score, round(total_optionality_score, 6) as total_optionality_score, round(combined_bp, 4) as combined_bp, round(combined_bp_DoW, 4) as combined_bp_DoW, round(combined_bp_DoW_DTG, 4) as combined_bp_DoW_DTG
  from total_BP),

directionality as (
  select route, left(route, 3) as base, right(route, 3) as dest, avg(case when orgcity = left(route, 3) then 1.0 else 0.0 end) as prop_from_base, avg(case when orgcity = right(route, 3) then 1.0 else 0.0 end) as prop_from_dest
  from data_prod.silver_sanezdb.psrhistory
  where 1=1
  and to_date(left(flightsector, 8), 'yyyyMMdd') between '2024-04-01' and '2024-06-30'
  group by route),

route_directionality as (
  select a.*, b.base, b.dest, b.prop_from_base, b.prop_from_dest
  from curated_BP a
  left join directionality b
  on a.route=b.route
),

onsale_dates as (
  select flightkey, first(onsale_dt) as onsale_dt
  from data_experience_commercial.cbt_0923_segmentfinder.dimensions_history
  where flight_dt between '2024-04-01' and '2024-06-30'
  group by flightkey
),

sale_length as (
  select a.*, datediff(a.flight_dt, b.onsale_dt) as sale_length, round((datediff(a.charge_dt, b.onsale_dt)/datediff(a.flight_dt, b.onsale_dt)), 2) as sale_period_progress
  from route_directionality a
  left join onsale_dates b
  on a.flightkey=b.flightkey
),

flight_time(
  select flightkey, first(parentregion) as parentregion, first(routetype) as routetype, first(flight_dow) as flight_dow, first(flight_hod) as flight_hod
  from data_experience_commercial.cbt_0923_segmentfinder.dimensions_history
  where flight_dt between '2024-04-01' and '2024-06-30'
  group by flightkey
),

final_time as (
  select a.flightkey, b.time_quality_score
  from flight_time a
  left join scaled_quality_scores b
  on a.parentregion=b.parentregion and a.routetype=b.routetype and a.flight_dow=b.flight_dow and a.flight_hod=b.flight_hod
),

final_table as (
  select a.*, round(b.time_quality_score, 4) as time_quality_score
  from sale_length a
  left join final_time b
  on a.flightkey=b.flightkey
)

select * from final_table
where 1=1
and dtg <=252
and dtg >=4
and flight_dt >= '2024-04-01'
and flight_dt <= '2024-06-30'
limit 1048000